# Выражения и условия в SQL. CRUD-операции

### Основные элементы языка SQL

СУБД управления базами данных PostgreSQL обрабатывает команды на языке SQL. Каждая команда складывается из элементов, как предложение из слов. Давайте разберём эти элементы, чтобы понимать, как СУБД читает и выполняет наши инструкции.

**Язык SQL включает семь ключевых элементов:**

1. **Ключевые слова** — это слова, которые зарезервировала система, чтобы использовать их как команды или их части. Они определяют действие, которое нужно выполнить. Например, ключевое слово `SELECT` говорить системе извлечить данные, `FROM` — указывает на источник, а `WHERE` — вводит условие отбора.

    Давайте посмотрим, как СУБД выполняет запрос c тремя ключевыми словами `SELECT`, `FROM` и `WHERE`:

In [ ]:
%%sql
SELECT make, year FROM car WHERE year > 2015 LIMIT 3;

Как мы видим, система взяла таблицу `car`, проверила условие `year > 2015` для каждой строки и вывела марку и год только для тех автомобилей, которые прошли фильтр. Этот пример показывает, как ключевые слова влияют на логику выборки значений.

2. **Идентификаторы** — это имена, которые мы даём объектам в базе: таблицам, столбцам и представлениям, чтобы СУБД могла искать по ним данные.
    
    Давайте проверим, какие столбцы есть в таблице `car`:

In [ ]:
%%sql
SELECT column_name, data_type FROM information_schema.columns
WHERE table_name = 'car' ORDER BY ordinal_position;

Система вывела точную структуру таблицу с шестью столбцами:
- `cid` — целое число;
- `make` — текст;
- `year` — целое число;
- `mileage` — целое число;
- `clsid` — текст;
- `res_number` — целое число. 

Теперь мы знаем, какие идентификаторы мы можем использовать. Давайте попробуем написать запрос, чтобы узнать данные о пробеге:

In [ ]:
%%sql
SELECT cid, make, mileage FROM car WHERE mileage < 50000 LIMIT 3;

3. **Выражения** — это комбинации значений, имен столбцов, функций и операторов, которые вычисляют, чтобы получить одно конкретное значение или набор данных.

    Давайте напишем выражение, которое переводит пробег из миль в километры:

In [ ]:
%%sql
SELECT make, mileage, mileage * 1.60934 AS mileage_km FROM car LIMIT 3;

Как мы видим, система взяла значение `mileage` для каждого автомобиля, умножила его на `1.60934` и вывела результат в новом столбце `mileage_km`. Это и другие выражения позволяют динамически обрабатывать данные, но не изменять саму таблицу.

4. **Условия** — это выражение, которое система проверяет как истину или ложь. Условия используют, чтобы фильтровать строки. Условия используют операторы сравнения `>`, `<`, `=`, а также логические связки `AND` и `OR`.

    Давайте попробуем написать условие с оператором `AND`, чтобы отобрать строки с автомобилями, которые выпустили между `2010` и `2020` годом, и чем пробег больше `100000`:

In [ ]:
%%sql
SELECT make, year, mileage FROM car
WHERE year BETWEEN 2010 AND 2020 AND mileage > 100000
LIMIT 3;

Система проверила каждую строку таблицы `car` и не включила ни одной строки в результат, потому что ни один автомобиль не соответствовал условию, которое мы задали.

5. **Переменные** — это объекты-контейнеры, которые используют, что временно хранить, изменять и передавать данные в скриптах, процедурах и функциях. Например, числа, строки и даты. 

6. **Комментарии** — это текст, который пишут разработчики, чтобы пояснить логику кода. Когда СУБД выполняет код, она игнорирует комментарии.

    Давайте посмотрим, как выглядят комментарии в SQL-коде:

In [ ]:
%%sql
-- Это однострочный комментарий. Его используют, чтобы прокомментировать одну строку.
SELECT * FROM car LIMIT 1;

/*
Это многострочный комментарий.
Его используют, чтобы прокомментировать блоки кода.
*/

Система выполнила только команду `SELECT * FROM car LIMIT 1` и вывела только одну полную строку из таблицы.

7. **NULL** — это специальное значение, которое означает отсутствие значения. Если вы читали первый конспект, то вы знаете, что это не ноль и не пустая строка. `NULL` показывает, что о данных нет информации, или что их нельзя применить. Напомню, чтобы проверить данные на значение `NULL`, используют специальные операторы `IS NULL` и `IS NOT NULL`.

Давайте напишем запрос и проверим, есть ли автомобили без номера резервации:

In [ ]:
%%sql
SELECT cid, make, res_number FROM car WHERE res_number IS NULL LIMIT 3;

Как мы видим, система не нашла автомобилей, у которых в столбце `res_number` есть `NULL`. Это означает, что нет никаких данных о том, есть ли бронь на автомобилях.

### Идентификаторы в SQL (1/3)

Мы разобрали основные элементы SQL и увидели, как СУБД работает с ключевыми словами и значениями. Теперь сосредоточимся на **идентификаторах, или именах объектов**. Мы уже использовали простые имена. Например, `car` и `mileage`. Но в реальной записи правила их записи сложнее. Давайте разберёмся, как система разрешает имена и как мы можем на них ссылаться. 

**Идентификатор** — это имя, которое мы даём объекту базы данных. Например, таблице, столбцу или представлению. СУБД использует идентификаторы, чтобы находить нужные данные среди таблиц и столбцов.

SQL позволяет записывать идентификаторы в разных форматах, а СУБД интепретирует их по строгим правилам.

**Давайте изучим четыре формата записи идентификаторов:**

1. **Простые идентификаторы** — это стандартные имена объектов, которые СУБД считает независимыми от регистра и приводит к нижнему регистру. Например, мы можем писать как `make`, так и `MAKE` или `Make`. СУБД распознает их как `make`.

In [ ]:
%%sql
SELECT make, MAKE, Make FROM car;

Как мы видим, СУБД создала три отдельные колонки и пронумеровала их. Значение в каждой из колонок одинаковое, потому что мы использовали простые идентификаторы. 

2. **Квалифицированные идентификаторы** — это полные имена, которые указывают путь к объекту через точку в формате `таблица.столбец`. Они устраняют неоднозначность, когда  в разных таблицах запроса встречаются одинаковые имена атрибутов.

    Давайте создадим простую вторую таблицу и испольуем полные имена, чтобы различить столбцы `cid`:

In [ ]:
%%sql
CREATE TEMP TABLE colors (cid INT, color TEXT);
INSERT INTO colors VALUES (1, 'Red'), (2, 'Blue');

SELECT car.cid, car.make, colors.color
FROM car
JOIN colors ON car.cid = colors.cid
WHERE car.cid <= 2;

В запросе столбец `cid` есть и в `car`, и в `colors`. Мы используем полные имена `car.cid` и `colors.cid` в условии `ON`. В `SELECT` мы явно указываем `car.cid`, чтобы система вывела идентификатор из таблицы автомобилей. Это показывает, как полные имена помогают избежать ошибок в данных.

3. **Разграниченные идентификаторы** — это имена, которые мы заключаем в двойные кавычки. СУБД сохраняет их регистр. Без кавычек СУБД не найдёт объект с именем в верхнем регистре. 

    Давайте напишем запрос и сравним закавыченные и незакавыченные идентификаторы:

In [ ]:
%%sql
SELECT cid, "cid", "CID" FROM car LIMIT 1;

СУБД нашла столбец `cid` для первых двух запросов, но для `"CID"` она выдала ошибку, потому что в таблице `car` нет столбца с именем в верхнем регистре. Кавычки устанавливают правила учета регистра для системы

4. **Идентификатор `*`, или звезда,** — это специальный символ `*`, который заменяет собой полный список всех атрибутов отношения. Он работает как сокращённая запись, которая позволяет выбрать все доступные столбцы. Например, `*` означает все столбцы всех таблиц в запросе, а `car.*` — все столбцы таблицы `car`.

    Давайте напишем запрос, в котором используем `*` и `car.*` 

In [ ]:
%%sql
SELECT * FROM car LIMIT 2;

Этот запрос возвращает все шесть столбцов таблицы `car`: `cid`, `make`, `year`, `mileage`, `clsid` и `res_number`. Звезда автоматически подставляет их вместо того, чтобы явно перечислять.

Теперь давайте создадим вторую таблицу, чтобы показать, как работает `car.*` в запросе, где соединим две таблицы. Итоговая таблица будет содержать информацию о классе автомобиля:

In [ ]:
%%sql
-- Создадим временную таблицу `class_info` для демонстрации
CREATE TEMP TABLE class_info AS
SELECT DISTINCT clsid, 
       CASE clsid 
           WHEN 'eco' THEN 'Economy' 
           WHEN 'lux' THEN 'Luxury' 
           ELSE 'Standard' 
       END AS description
FROM car 
WHERE clsid IS NOT NULL;

-- Теперь используем `car.*` для выбора всех столбцов из `car` класса и добавим описание класса
SELECT car.*, class_info.description
FROM car
JOIN class_info ON car.clsid = class_info.clsid
WHERE car.clsid IS NOT NULL
LIMIT 3;

В этом запросе `car.*` разворачивается во все столбцы основной таблицы. К ним добавляется только один столбец `description` из таблицы `class_info`, которую мы присоединили. Это позволяет быстро получить все данные из основной таблицы с дополнительной информацией из связанной таблицы. Кроме того, мы не перечисляли каждый столбец таблицы `car`.

Важно отметить, что идентификатор `*` может вернуть много лишних данных и снизить производительность базы данных, если в таблице много столбцов. Поэтому её нужно использовать аккуратно.

### Идентификаторы в SQL (2/3)

Мы разобрались, как СУБД различается идентифкаторы таблиц и столбцов. Но в болших базах данных таблицы организуют в схемы, или логические пространства, которые помогают структурировать объекты. Сейчас мы узнаем, как указывать схемы явно, и как они помогают искать таблицы.

Вспомним, что **схемы — это формальное описание структуры базы данных.** Теперь мы можем дополнить это определение и сказать, что **схемы — ещё и именованный контейнер, который хранит объекты базы данных.** Например, таблицы, представления и функции. Помимо того, что схема хранит эти объекты, она упорядочивает их и позволяет управлять доступом к этим объектам.

Например, в PostgreSQL схема по умолчанию имеет имя `public`. Когда мы пишем запрос `SELECT * FROM car`, СУБД ищет таблицу `car` сначала в схеме `public`, а затем по пути поиска `search_path`.

Если нам нужно обратиться к таблице из конкретной схемы, мы указываем её имя перед именем таблицы, но разделяем точкой: `имя_схемы.имя_таблицы`. Это правило работает как для представлений, так и для ссылок на столбцы.

Давайте проверим, в какой схеме находится наша таблица `car`, и попробуем явно указать схему в запросе:

In [ ]:
%%sql
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_name = 'car';

Мы видим, что схема, которая хранит нашу таблицу `car`, называется `public`. Теперь напишем запрос и укажем схему явно:

In [ ]:
%%sql
SELECT public.car.cid, public.car.make
FROM public.car
WHERE public.car.year > 2015
LIMIT 2;

Мы увидели, что СУБД корректно обработала полное имя `public.car`. Она нашла таблицу и вернула нам данные. Несмотря на то, что СУБД нашла бы таблицу и без `public.`, такой синтаксис становится обязательным, если в базе есть несколько таблиц `car` в разных схемах, иля путь поиска таблицы не включачет схему по умолчанию. 

СУБД позволяет определить, где искать таблицы и с помощью параметра `search_path`. Давайте посмотрим текущий путь поиска для нашего подключения:

In [ ]:
%%sql
SHOW search_path;

СУБД вернула нам список схем, которые обычно проверяют по порядку, когда встречает имя таблицы без схемы. Первым значением идёт `$user`, то есть схема с именем текущего пользователя, а затем `public`. Теперь мы лучше понимаем, почему наш запрос `SELECT * FROM car` работает — СУБД находит таблицу в схеме `public`.

Схемы позволяют нескольким пользователям или приложениям работать в базе данных и не мешать друг другу. Например, бухгалтерия может хранить таблицы в схеме `accounting`, а отдел продаж — в схеме `sales`. Когда запрос явно указывает `accounting.invoices`, СУБД точно знает, какую таблицу использовать.  

Давайте создадим временную схему и посмотрим, как система различает одноимённые таблицы в разных схемах:

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS demo;
CREATE TABLE demo.car AS SELECT * FROM car LIMIT 1;

SELECT 'public' AS schema_name, cid, make FROM public.car WHERE cid = 1
UNION ALL
SELECT 'demo' AS schema_name, cid, make FROM demo.car;

Мы создали новую схему `demo`, которая копирует одну строку из `public.car` в таблицу `demo.car`. После этого запрос обращается к двум разным схемам и показывает, что система может содержать разные данные в одноимённых таблицах под разными схемами. Именно поэтому в сложных окружениях необходимо явно указывать схемы.